In [1]:
import numpy as np
import pandas as pd
import sqlite3

In [2]:
conn = sqlite3.connect("../Nexus.db")
notifications = pd.read_sql_query("SELECT * FROM notifications", conn)
stock_prices = pd.read_sql_query("SELECT * FROM stock_prices", conn)
osbx = pd.read_sql_query("SELECT * FROM osbx", conn)
conn.close()

In [3]:
notifications['date'] = pd.to_datetime(
    notifications['date'],
    format="%d.%m.%Y %H:%M"
)
notifications.head()

,id,company_name,date,market,issuer_id,link,message,pdf_text
0,1,Ayfie International AS.json,2025-09-04 09:05:00,MERK,AIX,message_654625,Godthåb Holding AS has today bought 105.000 s...,
1,2,Ayfie International AS.json,2025-08-28 10:22:00,MERK,AIX,message_654077,HAAS AS has today bought 150.000 shares in Ayf...,
2,3,Ayfie International AS.json,2025-05-09 18:17:00,MERK,AIX,message_645945,Reference is made to the stock exchange announ...,KRT-1500 Skjema for melding om transaksjoner u...
3,4,Ayfie International AS.json,2025-01-15 15:02:00,MERK,AIX,message_636562,"On 30 December 2024/14th January 2025, a demer...",
4,5,Ayfie International AS.json,2024-12-30 16:12:00,MERK,AIX,message_635776,HAAS AS has today bought 33.343 shares in Ayfi...,


In [4]:
stock_prices['Date'] = pd.to_datetime(stock_prices['Date'])
stock_prices['ticker'] = stock_prices['ticker'].str.replace('.OL', '', regex=False)
stock_prices.head()

,Date,Close,High,Low,Open,Volume,ticker,Adj Close
0,2020-07-07,20.774784,21.174299,17.418858,17.578665,867144.0,AIX,None
1,2020-07-08,23.891003,23.891003,21.178295,21.917398,530604.0,AIX,None
2,2020-07-09,26.607704,27.966057,24.610130,25.169451,458413.0,AIX,None
3,2020-07-10,25.249353,28.769081,23.891003,27.702377,270235.0,AIX,None
4,2020-07-13,23.970905,26.367996,22.372845,25.968481,228139.0,AIX,None


In [5]:
osbx['Date'] = pd.to_datetime(osbx['Date'])
osbx.head()

,Date,Close,High,Low,Open,Volume,ticker
0,2014-11-19,602.340027,604.630005,599.460022,600.890015,74593600,OSEBX
1,2014-11-20,600.349976,604.770020,597.929993,602.349976,79070200,OSEBX
2,2014-11-21,606.570007,608.559998,600.299988,600.349976,123613000,OSEBX
3,2014-11-24,607.510010,609.890015,606.159973,606.580017,84988700,OSEBX
4,2014-11-25,606.119995,607.950012,602.599976,607.510010,85027000,OSEBX


In [6]:
notifications = notifications.sort_values(['issuer_id', 'date'])
stock_prices = stock_prices.sort_values(['ticker', 'Date'])
osbx = osbx.sort_values('Date')

In [7]:
# Stock returns per ticker
stock_prices['ret'] = stock_prices.groupby('ticker')['Close'].pct_change()
stock_prices['log_ret'] = np.log1p(stock_prices['ret'])

# OSEBX returns
osbx['ret'] = osbx['Close'].pct_change()
osbx['log_ret'] = np.log1p(osbx['ret'])


/home/imre/wolf_gang/.venv/lib/python3.13/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [8]:
prices_merged = stock_prices.merge(
    osbx[['Date', 'log_ret']].rename(columns={'log_ret': 'mkt_log_ret'}),
    on='Date',
    how='left'
)

prices_merged['abn_ret'] = prices_merged['log_ret'] - prices_merged['mkt_log_ret']

In [9]:

def align_to_ticker_trading_day(ticker, notif_date):
    df = prices_merged[prices_merged['ticker'] == ticker]

    candidates = df[df['Date'] >= notif_date]['Date']
    if len(candidates) == 0:
        return None
    return candidates.iloc[0]

notifications['event_date'] = notifications.apply(
    lambda row: align_to_ticker_trading_day(row['issuer_id'], row['date']),
    axis=1
)

notifications = notifications.dropna(subset=['event_date'])


In [10]:
def compute_car(ticker, event_date, H):
    # Select data for the correct stock
    df = prices_merged[prices_merged['ticker'] == ticker]

    # Must contain the event date
    df = df.reset_index(drop=True)
    event_rows = df[df['Date'] == event_date]

    if len(event_rows) == 0:
        return np.nan

    event_idx = event_rows.index[0]

    # Slice days after event
    window = df.loc[event_idx+1 : event_idx+H, 'abn_ret']

    if len(window) < H:
        return np.nan

    # Sum log abnormal returns = cumulative abnormal return
    return window.sum()

In [11]:
for H in [42]: # was originaly doing multiple periods but decided to change :)
    notifications[f'car_{H}'] = notifications.apply(
        lambda row: compute_car(row['issuer_id'], row['event_date'], H),
        axis=1
    )

In [12]:
notifications.head()

,id,company_name,date,market,issuer_id,link,message,pdf_text,event_date,car_42
15938,15939,2020 Bulkers Ltd..json,2019-08-14 13:35:00,XOAX,2020,message_482838,"Olav Eikrem, Chief Technical Officer in 2020 B...",,2019-08-15,0.009525
15937,15938,2020 Bulkers Ltd..json,2019-08-21 17:04:00,XOAX,2020,message_483505,"Jeremy Kramer, Director of 2020 Bulkers Ltd., ...",,2019-08-22,0.033167
15936,15937,2020 Bulkers Ltd..json,2019-08-22 17:04:00,XOAX,2020,message_483636,"Jeremy Kramer, Director of 2020 Bulkers Ltd., ...",,2019-08-23,0.026759
15935,15936,2020 Bulkers Ltd..json,2019-11-26 13:46:00,XOAX,2020,message_490227,"Vidar Hasund, Chief Financial Officer of 2020 ...",,2019-11-27,-0.050793
15934,15935,2020 Bulkers Ltd..json,2019-11-27 17:48:00,XOAX,2020,message_490377,"Jeremy Kramer, Director of 2020 Bulkers Ltd., ...",,2019-11-28,-0.103633


In [13]:
#print(notifications["car_10"].isna().sum())
#print(notifications["car_10"].count())

In [14]:
notifications.to_csv("car.csv", index=False, encoding="utf-8")